In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
zip_path = '/content/drive/MyDrive/archive (2).zip'

extract_path = '/content/extracted_files/'

import zipfile
import os

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")


Files extracted to: /content/extracted_files/


In [3]:
import os

extracted_files = os.listdir(extract_path)
print("Extracted files:", extracted_files)

Extracted files: ['The IQ-OTHNCCD lung cancer dataset', 'Test cases']


In [4]:
pip install split-folders

In [5]:
import splitfolders

# Path to the original dataset
input_folder = '/content/extracted_files/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset'

# Path to save the split dataset
output_folder = '/content/split_dataset'

# Split the dataset into train and validation sets (80% train, 20% validation)
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.2))

Copying files: 1097 files [00:00, 1805.21 files/s]


In [6]:
!pip install torch torchvision torchaudio
!pip install matplotlib numpy

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

In [8]:
# Define AlexNet model
class AlexNet(nn.Module):
    def __init__(self, num_classes=3):  # Change num_classes to match your dataset
        super(AlexNet, self).__init__()
        self.model = models.alexnet(pretrained=True)
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [21]:
# Define data transformations (if not already defined)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([  # Use 'val' instead of 'valid'
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Path to the split dataset
data_dir = '/content/split_dataset'

# Load the datasets
image_datasets = {
    x: ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
    for x in ['train', 'val']  # Use 'val' instead of 'valid'
}

# Create data loaders
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x == 'train'), num_workers=4)
    for x in ['train', 'val']  # Use 'val' instead of 'valid'
}

# Get dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}  # Use 'val' instead of 'valid'
class_names = image_datasets['train'].classes

print(f"Class names: {class_names}")
print(f"Training set size: {dataset_sizes['train']}")
print(f"Validation set size: {dataset_sizes['val']}")  # Use 'val' instead of 'valid'

Class names: ['Bengin cases', 'Malignant cases', 'Normal cases']
Training set size: 876
Validation set size: 221


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [19]:
import os
print(os.listdir('/content/split_dataset'))

['train', 'val']


In [20]:
# Path to the split dataset
data_dir = '/content/split_dataset'

# Load the datasets
image_datasets = {
    x: ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
    for x in ['train', 'val']  # Use 'val' instead of 'valid'
}

# Create data loaders
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in ['train', 'val']  # Use 'val' instead of 'valid'
}

# Get dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}  # Use 'val' instead of 'valid'
class_names = image_datasets['train'].classes

print(f"Class names: {class_names}")
print(f"Training set size: {dataset_sizes['train']}")
print(f"Validation set size: {dataset_sizes['val']}")  # Use 'val' instead of 'valid'

KeyError: 'val'

In [22]:
# Initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=len(class_names)).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 139MB/s]


In [24]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print('Training complete')

Epoch 0/9
----------
train Loss: 0.4375 Acc: 0.8311
val Loss: 0.3564 Acc: 0.8688
Epoch 1/9
----------
train Loss: 0.4691 Acc: 0.8208
val Loss: 0.3574 Acc: 0.8914
Epoch 2/9
----------
train Loss: 0.3728 Acc: 0.8527
val Loss: 0.5684 Acc: 0.7692
Epoch 3/9
----------
train Loss: 0.4019 Acc: 0.8322
val Loss: 0.2562 Acc: 0.9050
Epoch 4/9
----------
train Loss: 0.3712 Acc: 0.8573
val Loss: 0.2965 Acc: 0.8778
Epoch 5/9
----------
train Loss: 0.3975 Acc: 0.8345
val Loss: 0.3638 Acc: 0.8552
Epoch 6/9
----------
train Loss: 0.3625 Acc: 0.8425
val Loss: 0.2377 Acc: 0.8914
Epoch 7/9
----------
train Loss: 0.3540 Acc: 0.8482
val Loss: 0.2738 Acc: 0.8733
Epoch 8/9
----------
train Loss: 0.3072 Acc: 0.8699
val Loss: 0.2900 Acc: 0.8869
Epoch 9/9
----------
train Loss: 0.3817 Acc: 0.8482
val Loss: 0.3270 Acc: 0.8778
Training complete


In [25]:
# Save the model
torch.save(model.state_dict(), 'alexnet_lung_cancer.pth')
print("Model saved as 'alexnet_lung_cancer.pth'")

Model saved as 'alexnet_lung_cancer.pth'


In [28]:
# Load the test dataset
test_dataset = ImageFolder(root=f'{data_dir}/val', transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Evaluate the model
model.eval()
running_corrects = 0

for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

    running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(test_dataset)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.8778


In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Define the AlexNet model class
class AlexNet(nn.Module):
    def __init__(self, num_classes=3):  # Change num_classes to match your dataset
        super(AlexNet, self).__init__()
        self.model = models.alexnet(pretrained=True)
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=3).to(device)  # Change num_classes to match your dataset

# Load the saved model weights
model.load_state_dict(torch.load('alexnet_lung_cancer.pth'))
model.eval()  # Set the model to evaluation mode

# Define the same transformations used during training
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the input image
image_path = '/content/Bengin case (101) (1).jpg'  # Replace with your image path
image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format

# Preprocess the image
input_tensor = data_transforms(image).unsqueeze(0)  # Add batch dimension
input_tensor = input_tensor.to(device)  # Move the input tensor to the appropriate device

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)
    _, predicted_class = torch.max(output, 1)

# Map the predicted class index to the class name
class_names = ['Benign', 'Malignant', 'Normal']  # Replace with your class names
predicted_class_name = class_names[predicted_class.item()]

print(f'Predicted class: {predicted_class_name}')

# Display the image
plt.imshow(image)
plt.title(f'Predicted: {predicted_class_name}')
plt.axis('off')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'